# Alexander Image Data

This plot currently shows 429 photos Alexander took between March 18th and August 10th.

In [36]:
from datetime import datetime

import pytz

from ipywidgets import HTML, HBox
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd

df = pd.read_csv("out.csv")



def parse_file_name(file_name):
    return file_name.split(".")[0].split("PXL_")[1]


class ESTLocalizer:

    def __init__(self):
        self.gmt = pytz.timezone("GMT")
        self.eastern = pytz.timezone("US/Eastern")

    def localize(self, stamp):
        date = datetime.strptime(stamp[:-3], "%Y%m%d_%H%M%S")
        dategmt = self.gmt.localize(date)
        return dategmt.astimezone(self.eastern)




def interactive_plot(df, fig, template, event="hover") :
    """
    Make a plot react on hover or click of a data point and update a HTML preview below it.
    **template** Should be a string and contain placeholders like {colname} to be replaced by the value
    of the corresponding data row.
    
    """

    html = HTML("")

    def update(trace, points, state):
        ind = points.point_inds[0]
        row = df.loc[ind].to_dict()
        
        localizer = ESTLocalizer()
        row["timestamp"] = localizer.localize(parse_file_name(row["fname"]))
        html.value = template.format(**row)

    fig = go.FigureWidget(data=fig.data, layout=fig.layout)

    if event == "hover" :
        fig.data[0].on_hover(update)
    else :
        fig.data[0].on_click(update)

    return HBox([fig, html])

plot = go.Figure(
    data=go.Scattergeo(
        lon=df["longitude"],
        lat=df["latitude"],
        text=df["fname"],
    ),
)
plot.update_layout(
    geo = dict(
        scope = "usa",
        showland = True,
        landcolor = "rgb(150, 150, 150)",
        showsubunits = True,
    ),
    height=800,
    width=1200,
)

template = """
<img style='max-width: 450px;' src='./data/{fname}'>
<p>{timestamp}</p>
"""
interactive_plot(df, plot, template, event="on_click")

    'data': [{'lat': array([39.149097, 39.149315, 39.149097, ..., 39.14909 , 39.…

It might be easier to use the zoom tool in this plot:


In [34]:
plot2 = px.scatter(
    df,
    x="longitude",
    y="latitude",
    height=800,
    width=1200,
)
interactive_plot(df, plot2, template, event="on_click")

    'data': [{'hovertemplate': 'longitude=%{x}<br>latitude=%{y}<extra></extra>',…